# Predict Future Sales


### Description
This challenge serves as final project for the "How to win a data science competition" Coursera course.
In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 
We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

다음 달에 모든 제품 및 상점의 총 판매량을 예측하도록 요청하고 있습니다. 이 경쟁을 해결하면 데이터 과학 기술을 적용하고 향상시킬 수 있습니다.

### Objective
- 주어진 데이터는  January 2013 to October 2015임
- 구할려고 하는 데이터는 shops and products for November 2015
- 즉 각 상점에서 각 아이템이 한달동안 얼마나 팔렸는지를 확인할 필요가 있음

### 검토 필드
- date_block_num - a consecutive month number, used for convenience 의미 없는 필드일 것으로 생각됨
- item_name - name of item, 아이템 이름과 판매간의 상관관계가 있을까?
- shop_name - name of shop, 가게 이름과 판매간의 상관관계가 있을까? 

### 가설
- Shop ID와 특정 제품 판매(또는 카테고리)간의 상관관계가 존재하는가? --> Shop 정보와 제품판매 개수(또는 카테고리) 정보를 Heat Map으로 보자
- 각 상점별, 아이템별, 카테고리별로 월단위 판매에 대한 고려가 필요함
- 소프트웨어 판매의 계절적 또는 월단위 요인이 있는가 -> November은 겨울이므로 겨울에 제품이 많이 팔리는 것에 대한 변수 설정이 필요함
- 제품명 또는 특징을 나타내는 제품들(DVD 나 할인등)이 월별 판매에 영향을 미칠 수 있는가?
- 가격은 월별로 또는 가게별로 상이할 수 있음 이럴 경우 가격을 예측하는 모듈을 새로 만들 필요가 있음


### TODO
- [x] train 데이터를 월별로 정리할 것
- [ ] 가격이 월별로 또는 가게별로 변화하는 지 확인 -> 가격의 변화를 예측하는 모듈이 따로 필요할 수 있음

### Findings
- Item에 대한 가격은 같은 달 같은 가게임에도 차이가 있음

In [79]:
import pandas as pd

In [80]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")

train_df = pd.read_csv("data/sales_train.csv")
test_df = pd.read_csv("data/test.csv")

In [81]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [82]:
date_df = pd.DataFrame(train_df["date"].str.split(".").tolist(),  columns = ['day','month','year'])
date_df.head()

,day,month,year
0,02,01,2013
1,03,01,2013
2,05,01,2013
3,06,01,2013
4,15,01,2013


In [85]:
pre_df = pd.merge(train_df, date_df, left_index=True, right_index=True)
del pre_df["date"]
del pre_df["date_block_num"]
pre_df.head()

,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,59,22154,999.00,1.0,02,01,2013
1,25,2552,899.00,1.0,03,01,2013
2,25,2552,899.00,-1.0,05,01,2013
3,25,2554,1709.05,1.0,06,01,2013
4,25,2555,1099.00,1.0,15,01,2013


In [87]:
pre_df.groupby(["shop_id","item_id","item_price"]).sum()

item_cnt_day
shop_id item_id item_price              
0       30      265.00              31.0
        31      434.00              11.0
        32      221.00              16.0
        33      347.00               6.0
        35      247.00              15.0
        36      357.00               1.0
        40      127.00               1.0
        42      127.00               1.0
        43      221.00               1.0
        49      127.00               2.0
        51      127.00               4.0
                130.00               1.0
        57      167.00               1.0
        59      110.00               1.0
        61      195.00               1.0
        75      76.00                1.0
        85      190.00               1.0
        88      76.00                1.0
        95      193.00               1.0
        96      70.00                1.0
        97      67.00                1.0
        98      268.00              30.0
        111     89.00                1.0
                99.00                1.0
        115     99.00                1.0
        136     89.00                1.0
        149     99.00                3.0
        151     75.00                1.0
        152     97.00                1.0
        153     258.00               3.0
...                                  ...
59      22102   1250.00              4.0
        22104   249.00               1.0
        22105   199.00               2.0
                299.00               1.0
        22106   299.00               1.0
        22114   28.00                1.0
                349.00               1.0
        22115   149.00               1.0
        22118   99.00                1.0
        22126   149.00               2.0
                249.00               2.0
        22130   349.00               4.0
        22132   149.00               2.0
        22134   399.00               3.0
        22139   499.00               1.0
                699.00               1.0
        22141   399.00               2.0
        22143   349.00               8.0
        22144   1499.00              1.0
        22145   549.00               4.0
        22150   199.00               2.0
        22151   399.00               3.0
        22153   135.33               1.0
        22154   999.00               1.0
        22155   149.00               1.0
        22162   349.00               3.0
                399.00              13.0
        22164   699.00               3.0
                749.00               3.0
        22167   299.00               4.0

[737820 rows x 1 columns]